In [9]:
import spacy
from itertools import tee, islice, chain

In [38]:
'''
function that receive a iterable (list) and return the actual, before and after
'''
def previous_actual_next_iterable(l):
    prevs, items, nexts = tee(l, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)

In [39]:
rawText = "El acuario es chiquito chiquito. Hay otro más grande pero ... fuera del ... de la ciudad, y no teníamos ganas de alejarnos mucho. Hay una sección con ... con pececitos, cangrejos y bichos por el estilo, todos de la zona. Y otra sección con especies recogidas, ya sea porque los ... las abandonaron, o porque están lesionadas y no pueden ser devueltas al ecosistema. Entre ellas había una tortuga ... una tortuga de agua, grande, tendría sus 50 cm de ... caparazón, y la pobre  era  cieguita.  También  había  serpientes  y  lagartos,  y  uno  podía  interactuar  con  ellos.  Maddalena  dijo  “no, gracias”, pero yo dije “siiii” a todo, así que me vi envuelta por serpientes y lagartos. Maddalena me tomó varias fotos con su teléfono, y le di mi correo para que me las mande, así que estoy esperando esas fotos. Salimos del acuario, nos despedimos para siempre con un abrazo, y me volví al ... hotel. No tenía mucha idea como volver, pero el ... la intuición me llevó derechito y llegué cerca de las 4.00. Como era bien temprano, y no tenía ganas de salir otra vez (hacía muucho calor) me puse el traje de baño y bajé a la piscina. Me bañé un ratito, y luego, aprovechando el lindo día, me quedé un rato disfrutando el sol, cuidando de no quedarme dormida."

In [40]:
#Spacy configuration
#python3 -m spacy download en_core_web_sm #in terminal
spc = spacy.load("es_core_news_sm")

In [45]:
text = spc(" ".join(rawText.split()))

In [46]:
'''
function that iterate with all fields from spacy obtaining before, after and actual elements from every iteration conditioning with some predicates like
    - before and after element DET type and '...' text
    - before and after element ADP type and '...' text
    - before and after element DET and PRON type respectively and '...' text
    - just '...' text
    - actual and after element are equals
'''
def generate(text):
    o = [text[0]]

    for before, actual, after in previous_actual_next_iterable(text):

        if before is not None and after is not None:
            #print(before, before.pos_ , actual, after, after.pos_)
            if before.pos_ == "DET" and after.pos_ == "DET" and actual.text == "...":
                o.pop()
                continue

            if before.pos_ == "ADP" and after.pos_ == "ADP" and actual.text == "...":
                o.pop()
                continue

            if before.pos_ == "DET" and after.pos_ == "PRON" and actual.text == "...":
                o.pop()
                continue

            if actual.text == "...":
                continue

            if actual.text == after.text:
                continue

            o.append(actual)

    o.append(text[-1:])

    s = ""
    for i in o:
        s += (str(i.text) + " ")

    return s.replace(" . ", ". ").replace(" , ", ", ")

In [47]:
string = generate(text)

In [48]:
string

'El acuario es chiquito. Hay otro más grande pero fuera de la ciudad, y no teníamos ganas de alejarnos mucho. Hay una sección con pececitos, cangrejos y bichos por el estilo, todos de la zona. Y otra sección con especies recogidas, ya sea porque las abandonaron, o porque están lesionadas y no pueden ser devueltas al ecosistema. Entre ellas había una tortuga una tortuga de agua, grande, tendría sus 50 cm de caparazón, y la pobre era cieguita. También había serpientes y lagartos, y uno podía interactuar con ellos. Maddalena dijo “ no, gracias ”, pero yo dije “ siiii ” a todo, así que me vi envuelta por serpientes y lagartos. Maddalena me tomó varias fotos con su teléfono, y le di mi correo para que me las mande, así que estoy esperando esas fotos. Salimos del acuario, nos despedimos para siempre con un abrazo, y me volví al hotel. No tenía mucha idea como volver, pero la intuición me llevó derechito y llegué cerca de las 4.00. Como era bien temprano, y no tenía ganas de salir otra vez ( 